In [1]:
import pandas as pd

### Automatically Count Label

In [2]:
# data = pd.read_excel('data/keyword_extraction_eval_annotator.xlsx')
# data.head()

In [3]:
# # Function to extract keywords and create new DataFrame
# def extract_keywords(row):
#     keywords = []
#     for col in data.columns:
#         keywords.extend(row[col].split(', '))
#     return pd.Series({'index': row.name, 'keywords': keywords})

# # Apply the function to each row of the DataFrame
# new_data = data.apply(extract_keywords, axis=1).explode('keywords').reset_index(drop=True)

# print(new_data)


In [4]:
# # Function to determine labeling and not labeling counts for each keyword
# def determine_labeling(row):
#     keyword = row['keywords']
#     labeling = 0
#     not_labeling = 0
    
#     # Get the row from data DataFrame based on the index
#     data_row = data.iloc[row['index']]
    
#     # Iterate over each column (Responden) in the data DataFrame
#     for col in data.columns[2:]:
#         data_row[col] = data_row[col].replace(",", "") # list keyword tiap responden
#         if any(subset in word.split(' ') for word in data_row[col] for subset in keyword ):
#         # if any(keyword in word.split(", ") for word in data_row[col]):
#             labeling += 1
#         else:
#             not_labeling += 1
#     return pd.Series({'labeling': labeling, 'not_labeling': not_labeling})

# # Apply the function to each row of new_data DataFrame
# new_data[['labeling', 'not_labeling']] = new_data.apply(determine_labeling, axis=1)

# print(new_data)

In [5]:
# # hasile

# eval_count = new_data[["index", "keywords"]]
# eval_count.to_excel('data/keyword_extraction_eval_count_dummy.xlsx', index=False)

### Manually Count Label

In [6]:
data = pd.read_excel('data/keyword_extraction_eval_count.xlsx')
data = data[data['labeling'] >= 3]
data.head()

,index,keywords,Responden 1,Responden 2,Responden 3,Responden 4,Responden 5,labeling,not_labeling
0,0,automatically generating vocabulary problems,1.0,1.0,1.0,1.0,1.0,5,0
1,0,generate potential answers,1.0,1.0,1.0,1.0,1.0,5,0
5,0,automatically generating vocabulary problems,1.0,1.0,1.0,1.0,1.0,5,0
7,0,generate potential answers of high difficulty ...,1.0,1.0,1.0,1.0,1.0,5,0
8,0,inputting target words,0.0,1.0,1.0,0.0,1.0,3,2


### Fleiss Kappa Each Keywords

In [7]:
N = 1  # number of data
n = 5  # number of respondent
k = 2  # number of categories

list_k = []
list_pi = []
for index, row in data.iterrows():
    pi = (row['labeling']**2 + row['not_labeling']**2 - n) / (n*(n-1))
    labeling_pj = row['labeling'] / (N*n)
    not_labeling_pj = row['not_labeling'] / (N*n)
    p = 1 / N * pi
    pe = labeling_pj**2 + not_labeling_pj**2
    if pe == 1:
        k = 1  # Handle division by zero in the formula
    else:
        k = (p - pe) / (1 - pe)
    list_pi.append(pi)
    list_k.append(k)

data['pi'] = list_pi
data['k'] = list_k
data.head()


,index,keywords,Responden 1,Responden 2,Responden 3,Responden 4,Responden 5,labeling,not_labeling,pi,k
0,0,automatically generating vocabulary problems,1.0,1.0,1.0,1.0,1.0,5,0,1.0,1.00
1,0,generate potential answers,1.0,1.0,1.0,1.0,1.0,5,0,1.0,1.00
5,0,automatically generating vocabulary problems,1.0,1.0,1.0,1.0,1.0,5,0,1.0,1.00
7,0,generate potential answers of high difficulty ...,1.0,1.0,1.0,1.0,1.0,5,0,1.0,1.00
8,0,inputting target words,0.0,1.0,1.0,0.0,1.0,3,2,0.4,-0.25


### Fleiss Kappa All Keywords

In [8]:
unique_data = data.drop_duplicates() # get unique keywords
N = len(unique_data) # number of unique_data
n = 5 # number of responden
k = 2 # number of category

# Calculate pi for each row
list_pi = []
for index, row in unique_data.iterrows():
    n = row['labeling'] + row['not_labeling']
    pi = 1 / (n*(n-1)) * (row['labeling']**2 + row['not_labeling']**2 - n)
    list_pi.append(pi)

# Calculate pj
labeling_pj = 1 / (N*n) * unique_data['labeling'].sum()
not_labeling_pj = 1 / (N*n) * unique_data['not_labeling'].sum()

# Calculate p and pe
p = 1 / N * sum(list_pi)
pe = labeling_pj**2 + not_labeling_pj**2
print("p = " + str(p))
print("pe = " + str(pe))

# Calculate k
k = (p-pe) / (1-pe)
print("k = " + str(k))

p = 0.757471264367816
pe = 0.7577675019450685
k = -0.0012229472908516271


### Cohen's Kappa

In [11]:
# def observed_agreement(i, j):
#     confusion_matrix = pd.crosstab(data['Responden '+str(i)], data['Responden '+str(j)])
#     diagonal_sum = sum(confusion_matrix.iloc[i,i] for i in range(len(confusion_matrix)))
#     return diagonal_sum / len(data)

# def chance_agreement(i, j):
#     df = data[['Responden '+str(i), 'Responden '+str(j)]]
#     sum_rows = [sum(row) for row in df.values]
#     sum_columns = [sum(col) for col in df.values.transpose()]
#     p_i = sum([row * row for row in sum_rows]) / (sum(sum_rows) ** 2)
#     p_j = sum([col * col for col in sum_columns]) / (sum(sum_columns) ** 2)
#     return p_i * p_j

# def count_kappa(i, j):
#     # Calculate observed agreement
#     observed_agr = observed_agreement(i, j)
#     # Calculate chance agreement
#     chance_agr = chance_agreement(i, j)
#     # Calculate Cohen's Kappa
#     kappa = (observed_agr - chance_agr) / (1 - chance_agr)
#     return kappa
    
# for i in range(1, 6):  # Looping pertama dari 1 sampai 5
#     for j in range(i+1, 6):  # Looping kedua dari nilai i sampai 5
#         k = count_kappa(i, j)
#         print(f'nilai kappa responden {i} dengan responden {j} = {k}')